In [2]:
# ----------------------------------------------------------------------------
# The implementation is based on code provided by Snap Inc. under their own license:
# Copyright Snap Inc. 2023. All rights reserved.
# See Snap Inc.'s license for more details.
# ----------------------------------------------------------------------------

import os.path as osp
from ogb.nodeproppred import Evaluator, PygNodePropPredDataset
import torch
import torch.nn.functional as F

from tqdm import tqdm

from torch_geometric.loader import NeighborSampler
from torch_geometric.nn import SAGEConv
from torch.nn import Linear
from typing import Tuple, Union

import torch.nn.functional as F
from torch import Tensor
from torch_geometric.nn.dense.linear import Linear
from torch_geometric.typing import Adj, OptPairTensor, Size
import torch.utils.data as data_utils

import argparse

In [3]:
from torch_geometric.nn.models import DeepGraphInfomax

In [4]:
import wandb

In [5]:
wandb.init(
    project = "pretrain-mlpinit",
    dir = "./wandb",
    config = {
        "init_method": "mlp",
        "dataset_name": "ogbn-arxiv",
        "batch_size": 1024,
        "num_layers": 4,
        "hidden_channels": 512
    }
)

Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


In [6]:
from sklearn.metrics import f1_score
import os.path as osp
from ogb.nodeproppred import Evaluator, PygNodePropPredDataset
import torch
import numpy as np

from tqdm import tqdm

from torch_geometric.loader import NeighborSampler
from torch_geometric.nn import SAGEConv
from torch.nn import Linear
from typing import Tuple, Union

import torch.nn.functional as F
from torch import Tensor
from torch_geometric.nn.dense.linear import Linear
from torch_geometric.typing import OptPairTensor
import torch.utils.data as data_utils

from torch_geometric.nn.models import DeepGraphInfomax
import wandb
import argparse

import sys
from src.load_dataset import load_data

class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

args_dict = {"dataset": "Flickr"}
args = dotdict(args_dict)

if args.dataset == "Flickr":
    args.num_classes = 7
    args.num_feats = 500

elif args.dataset == "Reddit":
    args.num_classes = 41
    args.num_feats = 602

elif args.dataset == "Reddit2":
    args.num_classes = 41
    args.num_feats = 602

elif args.dataset == "ogbn-products":
    args.multi_label = False
    args.num_classes = 47
    args.num_feats = 100

elif args.dataset == "AmazonProducts":
    args.multi_label = True
    args.num_classes = 107
    args.num_feats = 200

elif args.dataset == "Yelp":
    args.multi_label = True
    args.num_classes = 100
    args.num_feats = 300

elif args.dataset == "ogbn-arxiv":
    args.num_feats = 128
    args.num_classes = 40
    args.N_nodes = 169343


num_feats = args.num_feats
num_classes = args.num_classes

wandb.init(
    project="pretrain-mlpinit",
    dir="./wandb",
    config={
        "dataset_name": "ogbn-arxiv",
        "batch_size": 1024,
        "num_layers": 4,
        "hidden_channels": 1024,
        "percent_corrupted": 0.25,
        "custom_step": 0
    }
)

dataset_dir = "./data"
num_workers = 4

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
root = osp.join(dataset_dir, args.dataset)

(data, x, y, split_masks, evaluator, processed_dir) = load_data(args.dataset, "./data")

print(evaluator)

train_idx = split_masks["train"]
split_idx = split_masks

x_train = data.x[split_masks["train"]]
y_train = data.y[split_masks["train"]].reshape(-1).type(torch.long)

print("data.x.shape:", data.x.shape)
print("data.y.shape:", data.y.shape)
print("data.x.type:", x.dtype)
print("data.y.type:", y.dtype)
print("x_train.shape:", x_train.shape)
print("y_train.shape:", y_train.shape)

y = data.y.squeeze().type(torch.long)

x_y_train_mlpinit = data_utils.TensorDataset(x_train, y_train)
x_y_all_mlpinit = data_utils.TensorDataset(x, y)

train_mlpinit_loader = data_utils.DataLoader(
    x_y_train_mlpinit,
    batch_size=wandb.config.batch_size,
    shuffle=True,
    num_workers=num_workers,
)
all_mlpinit_loader = data_utils.DataLoader(
    x_y_all_mlpinit,
    batch_size=wandb.config.batch_size,
    shuffle=False,
    num_workers=num_workers,
)


# class SAGEConv_PeerMLP(torch.nn.Module):
#     """
#     A PyTorch module implementing a simplified GraphSAGE convolution-like multilayer perceptron (MLP) layer.
#
#     This layer performs a linear transformation on the input node features, optionally normalizing
#     the output and adding a root weight.
#     """
#
#     def __init__(
#         self,
#         in_channels: Union[int, Tuple[int, int]],
#         out_channels: int,
#         normalize: bool = False,
#         root_weight: bool = True,
#         bias: bool = True,
#         **kwargs,
#     ):
#         super().__init__(**kwargs)
#
#         self.in_channels = in_channels
#         self.out_channels = out_channels
#         self.normalize = normalize
#         self.root_weight = root_weight
#
#         if isinstance(in_channels, int):
#             in_channels = (in_channels, in_channels)
#
#         self.lin_l = Linear(in_channels[0], out_channels, bias=bias)
#         if self.root_weight:
#             self.lin_r = Linear(in_channels[1], out_channels, bias=False)
#
#         self.reset_parameters()
#
#     def reset_parameters(self):
#         self.lin_l.reset_parameters()
#         if self.root_weight:
#             self.lin_r.reset_parameters()
#
#     def forward(self, x: Union[Tensor, OptPairTensor]) -> Tensor:
#         """"""
#         if isinstance(x, Tensor):
#             x: OptPairTensor = (x, x)
#
#         out = x[1]
#         out = self.lin_l(out)
#
#         x_r = x[1]
#         if self.root_weight and x_r is not None:
#             out += self.lin_r(x_r)
#
#         if self.normalize:
#             out = F.normalize(out, p=2.0, dim=-1)
#
#         return out

#
# class MLP(torch.nn.Module):
#     def __init__(self, in_channels, hidden_channels, out_channels, num_layers):
#         super().__init__()
#
#         self.num_layers = num_layers
#
#         self.convs = torch.nn.ModuleList()
#         self.convs.append(SAGEConv_PeerMLP(in_channels, hidden_channels))
#         for _ in range(num_layers - 2):
#             self.convs.append(SAGEConv_PeerMLP(hidden_channels, hidden_channels))
#         self.convs.append(SAGEConv_PeerMLP(hidden_channels, out_channels))
#
#     def reset_parameters(self):
#         for conv in self.convs:
#             conv.reset_parameters()
#
#     def forward(self, x):
#         for i in range(self.num_layers):
#             x_target = x
#             x = self.convs[i]((x, x_target))
#             if i != self.num_layers - 1:
#                 x = F.relu(x)
#         return x.log_softmax(dim=-1)


class SAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers):
        super().__init__()

        self.num_layers = num_layers

        self.convs = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels))
        self.convs.append(SAGEConv(hidden_channels, out_channels))

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()

    def forward(self, x, adjs):
        # `train_loader` computes the k-hop neighborhood of a batch of nodes,
        # and returns, for each layer, a bipartite graph object, holding the
        # bipartite edges `edge_index`, the index `e_id` of the original edges,
        # and the size/shape `size` of the bipartite graph.
        # Target nodes are also included in the source nodes so that one can
        # easily apply skip-connections or add self-loops.
        for i, (edge_index, _, size) in enumerate(adjs):
            x_target = x[: size[1]]  # Target nodes are always placed first.
            x = self.convs[i]((x, x_target), edge_index)
            if i != self.num_layers - 1:
                x = F.relu(x)
        return x.log_softmax(dim=-1)

    def inference(self, x_all):
        pbar = tqdm(total=x_all.size(0) * self.num_layers)
        pbar.set_description("Evaluating")

        # Compute representations of nodes layer by layer, using *all*
        # available edges. This leads to faster computation in contrast to
        # immediately computing the final representations of each batch.
        total_edges = 0
        for i in range(self.num_layers):
            xs = []
            for batch_size, n_id, adj in subgraph_loader:
                edge_index, _, size = adj.to(device)
                total_edges += edge_index.size(1)
                x = x_all[n_id].to(device)
                x_target = x[: size[1]]
                x = self.convs[i]((x, x_target), edge_index)
                if i != self.num_layers - 1:
                    x = F.relu(x)
                xs.append(x.cpu())

                pbar.update(batch_size)

            x_all = torch.cat(xs, dim=0)

        pbar.close()

        return x_all


# model_mlpinit = MLP(
#     in_channels=args.num_feats,
#     hidden_channels=wandb.config.hidden_channels,
#     out_channels=args.num_classes,
#     num_layers=wandb.config.num_layers,
# )
#
# model_mlpinit = model_mlpinit.to(device)
# optimizer_model_mlpinit = torch.optim.Adam(model_mlpinit.parameters(), lr=0.001, weight_decay=0.0)

## теперь нужно посмотреть, тренируется ли эта модель и как. найти где можно организовать графики
## найти как они измеряют и запустить тупо на всём датасете
## всунуть дги везде где можно чтобы проверить на всем чем можно

def index_corruption(x):
    num_nodes = x.size()[0]
    mask = torch.ones(num_nodes, num_feats)
    mask[:][torch.randperm(num_nodes)[:int(num_feats * wandb.config.percent_corrupted)]] = 0
    mask = mask.bool().to(device)

    x = torch.where(mask.bool(), x, torch.zeros_like(x))
    return x


def dropout_corruption(x, p=0.9):
    mask = torch.empty_like(x).bernoulli_(p)
    x = torch.where(mask.bool(), x, torch.zeros_like(x))
    return x


def summary(z, *args, **kwargs):
    return torch.sigmoid(z.mean(dim=0))


unsupervised_model = DeepGraphInfomax(hidden_channels=num_classes, encoder=model_mlpinit, summary=summary,
                                          corruption=dropout_corruption)
unsupervised_model.to(device)

def train_mlpinit():
    total_loss = 0
    unsupervised_model.train()
    for x, _ in tqdm(train_mlpinit_loader):
        x = x.to(device)

        optimizer_model_mlpinit.zero_grad()
        pos_z, neg_z, summary_value = unsupervised_model(x)
        loss = unsupervised_model.loss(pos_z, neg_z, summary_value)
        loss.backward()
        optimizer_model_mlpinit.step()

        total_loss += float(loss)

    loss_percent = total_loss / len(train_mlpinit_loader)
    wandb.log({"loss_dgi": loss_percent})
    unsupervised_model.eval()
    return loss_percent, 0


@torch.no_grad()
def test_mlpinit():
    model_mlpinit.eval()

    out_list = []
    y_list = []

    for x, y in tqdm(all_mlpinit_loader):
        x = x.to(device)
        y = y.to(device)
        out = model_mlpinit(x)
        out_list.append(out)
        y_list.append(y)

    out = torch.cat(out_list, dim=0)

    if evaluator is not None:
        y_true = y.unsqueeze(-1)
        y_pred = out.argmax(dim=-1, keepdim=True)

        train_acc = evaluator.eval(
            {
                "y_true": y_true[split_masks["train"]],
                "y_pred": y_pred[split_masks["train"]],
            }
        )["acc"]
        valid_acc = evaluator.eval(
            {
                "y_true": y_true[split_masks["valid"]],
                "y_pred": y_pred[split_masks["valid"]],
            }
        )["acc"]
        test_acc = evaluator.eval(
            {
                "y_true": y_true[split_masks["test"]],
                "y_pred": y_pred[split_masks["test"]],
            }
        )["acc"]
    else:

        if not args.multi_label:
            pred = out.argmax(dim=-1).to("cpu")
            y_true = y
            correct = pred.eq(y_true)
            train_acc = correct[split_masks["train"]].sum().item() / split_masks["train"].sum().item()
            valid_acc = correct[split_masks["valid"]].sum().item() / split_masks["valid"].sum().item()
            test_acc = correct[split_masks["test"]].sum().item() / split_masks["test"].sum().item()

        else:
            pred = (out > 0).float().numpy()
            y_true = y.numpy()
            # calculating F1 scores
            train_acc = (
                f1_score(
                    y_true[split_masks["train"]],
                    pred[split_masks["train"]],
                    average="micro",
                )
                if pred[split_masks["train"]].sum() > 0
                else 0
            )

            valid_acc = (
                f1_score(
                    y_true[split_masks["valid"]],
                    pred[split_masks["valid"]],
                    average="micro",
                )
                if pred[split_masks["valid"]].sum() > 0
                else 0
            )

            test_acc = (
                f1_score(
                    y_true[split_masks["test"]],
                    pred[split_masks["test"]],
                    average="micro",
                )
                if pred[split_masks["test"]].sum() > 0
                else 0
            )

    return train_acc, valid_acc, test_acc


train_loader = NeighborSampler(
    data.edge_index,
    node_idx=train_idx,
    sizes=[25, 10, 5, 5],
    batch_size=wandb.config.batch_size,
    shuffle=True,
    num_workers=num_workers,
)
subgraph_loader = NeighborSampler(
    data.edge_index,
    node_idx=None,
    sizes=[-1],
    batch_size=wandb.config.batch_size,
    shuffle=False,
    num_workers=num_workers,
)


model = SAGE(
    in_channels=args.num_feats,
    hidden_channels=wandb.config.hidden_channels,
    out_channels=args.num_classes,
    num_layers=wandb.config.num_layers,
)
model = model.to(device)


def train(epoch):
    model.train()

    pbar = tqdm(total=train_idx.size(0))
    pbar.set_description(f"Epoch {epoch:02d}")

    total_loss = total_correct = 0

    for batch_size, n_id, adjs in train_loader:
        # `adjs` holds a list of `(edge_index, e_id, size)` tuples.
        adjs = [adj.to(device) for adj in adjs]

        optimizer.zero_grad()

        out = model(x[n_id].to(device), adjs)
        loss = F.nll_loss(out, y[n_id[:batch_size]].to(device))
        loss.backward()
        optimizer.step()

        total_loss += float(loss)
        total_correct += int(out.argmax(dim=-1).eq(y[n_id[:batch_size]].to(device)).sum())
        pbar.update(batch_size)

    pbar.close()

    loss = total_loss / len(train_loader)
    approx_acc = total_correct / train_idx.size(0)

    return loss, approx_acc


@torch.no_grad()
def test():
    model.eval()

    out = model.inference(x)

    if evaluator is not None:
        y_true = y.unsqueeze(-1)
        y_pred = out.argmax(dim=-1, keepdim=True)

        train_acc = evaluator.eval(
            {
                "y_true": y_true[split_masks["train"]],
                "y_pred": y_pred[split_masks["train"]],
            }
        )["acc"]
        valid_acc = evaluator.eval(
            {
                "y_true": y_true[split_masks["valid"]],
                "y_pred": y_pred[split_masks["valid"]],
            }
        )["acc"]
        test_acc = evaluator.eval(
            {
                "y_true": y_true[split_masks["test"]],
                "y_pred": y_pred[split_masks["test"]],
            }
        )["acc"]
    else:

        if not args.multi_label:
            pred = out.argmax(dim=-1).to("cpu")
            y_true = y
            correct = pred.eq(y_true)
            train_acc = correct[split_masks["train"]].sum().item() / split_masks["train"].sum().item()
            valid_acc = correct[split_masks["valid"]].sum().item() / split_masks["valid"].sum().item()
            test_acc = correct[split_masks["test"]].sum().item() / split_masks["test"].sum().item()

        else:
            pred = (out > 0).float().numpy()
            y_true = y.numpy()
            # calculating F1 scores
            train_acc = (
                f1_score(
                    y_true[split_masks["train"]],
                    pred[split_masks["train"]],
                    average="micro",
                )
                if pred[split_masks["train"]].sum() > 0
                else 0
            )

            valid_acc = (
                f1_score(
                    y_true[split_masks["valid"]],
                    pred[split_masks["valid"]],
                    average="micro",
                )
                if pred[split_masks["valid"]].sum() > 0
                else 0
            )

            test_acc = (
                f1_score(
                    y_true[split_masks["test"]],
                    pred[split_masks["test"]],
                    average="micro",
                )
                if pred[split_masks["test"]].sum() > 0
                else 0
            )

    return train_acc, valid_acc, test_acc

Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


None
data.x.shape: torch.Size([89250, 500])
data.y.shape: torch.Size([89250])
data.x.type: torch.float32
data.y.type: torch.int64
x_train.shape: torch.Size([44625, 500])
y_train.shape: torch.Size([44625])
tensor([[-0.0128,  0.0421,  0.0364,  ...,  0.0424,  0.0158,  0.0115],
        [-0.0093,  0.0227, -0.0298,  ..., -0.0042, -0.0181, -0.0088],
        [-0.0110,  0.0058,  0.0076,  ..., -0.0210,  0.0016,  0.0373],
        ...,
        [-0.0256, -0.0169, -0.0420,  ...,  0.0438, -0.0195,  0.0335],
        [ 0.0268,  0.0138,  0.0186,  ..., -0.0118,  0.0143, -0.0208],
        [ 0.0096,  0.0129, -0.0172,  ..., -0.0008,  0.0169, -0.0246]])
tensor([ 0.0374, -0.0358,  0.0213,  ..., -0.0417,  0.0302, -0.0053])
tensor([[-0.0007,  0.0438, -0.0212,  ...,  0.0333,  0.0041, -0.0038],
        [-0.0025, -0.0230,  0.0309,  ...,  0.0389,  0.0248, -0.0349],
        [ 0.0080,  0.0194, -0.0033,  ...,  0.0176,  0.0274, -0.0144],
        ...,
        [ 0.0370, -0.0447, -0.0289,  ..., -0.0442, -0.0236,  0.0257],

In [ ]:
random_losses = []
random_test_accs = []

model.reset_parameters()

optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay = 0.0)

best_val_acc = final_test_acc = 0
for epoch in [1]:
    loss, acc = train(epoch)
    train_acc, val_acc, test_acc = test()
    print(f'Epoch {epoch:02d}, Train: {train_acc:.4f}, Val: {val_acc:.4f}, 'f'Test: {test_acc:.4f}')
    wandb.log({"loss": loss, "acc": test_acc})
    random_losses.append(loss)
    random_test_accs.append(test_acc)

Evaluating: 100%|██████████| 357000/357000 [00:20<00:00, 17087.13it/s]

Epoch 01, Train: 0.4638, Val: 0.4640, Test: 0.4635


In [ ]:
for epoch in range(1, 1):
    loss, acc = train_mlpinit()
for parametr in model_mlpinit.parameters():
    print(parametr)
torch.save(model_mlpinit.state_dict(), f'./model_mlpinit.pt' )
train_acc_init, val_acc_init, test_acc_init = test_mlpinit()
print("train_acc_init, val_acc_init, test_acc_init:", train_acc_init, val_acc_init, test_acc_init )

Parameter containing:
tensor([[-0.0128,  0.0421,  0.0364,  ...,  0.0424,  0.0158,  0.0115],
        [-0.0093,  0.0227, -0.0298,  ..., -0.0042, -0.0181, -0.0088],
        [-0.0110,  0.0058,  0.0076,  ..., -0.0210,  0.0016,  0.0373],
        ...,
        [-0.0256, -0.0169, -0.0420,  ...,  0.0438, -0.0195,  0.0335],
        [ 0.0268,  0.0138,  0.0186,  ..., -0.0118,  0.0143, -0.0208],
        [ 0.0096,  0.0129, -0.0172,  ..., -0.0008,  0.0169, -0.0246]],
       requires_grad=True)
Parameter containing:
tensor([ 0.0374, -0.0358,  0.0213,  ..., -0.0417,  0.0302, -0.0053],
       requires_grad=True)
Parameter containing:
tensor([[-0.0007,  0.0438, -0.0212,  ...,  0.0333,  0.0041, -0.0038],
        [-0.0025, -0.0230,  0.0309,  ...,  0.0389,  0.0248, -0.0349],
        [ 0.0080,  0.0194, -0.0033,  ...,  0.0176,  0.0274, -0.0144],
        ...,
        [ 0.0370, -0.0447, -0.0289,  ..., -0.0442, -0.0236,  0.0257],
        [ 0.0053, -0.0365, -0.0192,  ...,  0.0397,  0.0246,  0.0231],
        [-0.01

100%|██████████| 88/88 [00:08<00:00, 10.15it/s]


RuntimeError: The size of tensor a (89250) must match the size of tensor b (162) at non-singleton dimension 0

In [9]:
mlpinit_losses = []
mlpinit_test_accs = []

model.load_state_dict(torch.load( f'./model_mlpinit.pt'  ))
for parametr in model_mlpinit.parameters():
    print(parametr)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay = 0.0)

best_val_acc = final_test_acc = 0
for epoch in range(1, 1):
    loss, acc = train(epoch)
    train_acc, val_acc, test_acc = test()
    print(f'Epoch {epoch:02d}, Train: {train_acc:.4f}, Val: {val_acc:.4f}, 'f'Test: {test_acc:.4f}')
    wandb.log({"loss": loss, "acc": test_acc})
    mlpinit_losses.append(loss)
    mlpinit_test_accs.append(test_acc)

Parameter containing:
tensor([[-0.0128,  0.0421,  0.0364,  ...,  0.0424,  0.0158,  0.0115],
        [-0.0093,  0.0227, -0.0298,  ..., -0.0042, -0.0181, -0.0088],
        [-0.0110,  0.0058,  0.0076,  ..., -0.0210,  0.0016,  0.0373],
        ...,
        [-0.0256, -0.0169, -0.0420,  ...,  0.0438, -0.0195,  0.0335],
        [ 0.0268,  0.0138,  0.0186,  ..., -0.0118,  0.0143, -0.0208],
        [ 0.0096,  0.0129, -0.0172,  ..., -0.0008,  0.0169, -0.0246]],
       requires_grad=True)
Parameter containing:
tensor([ 0.0374, -0.0358,  0.0213,  ..., -0.0417,  0.0302, -0.0053],
       requires_grad=True)
Parameter containing:
tensor([[-0.0007,  0.0438, -0.0212,  ...,  0.0333,  0.0041, -0.0038],
        [-0.0025, -0.0230,  0.0309,  ...,  0.0389,  0.0248, -0.0349],
        [ 0.0080,  0.0194, -0.0033,  ...,  0.0176,  0.0274, -0.0144],
        ...,
        [ 0.0370, -0.0447, -0.0289,  ..., -0.0442, -0.0236,  0.0257],
        [ 0.0053, -0.0365, -0.0192,  ...,  0.0397,  0.0246,  0.0231],
        [-0.01

In [ ]:
import matplotlib.pyplot as plt

x=range( 0, 3 )

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 3))

ax1.plot(x, random_losses[:3], label='random')
ax1.plot(x, mlpinit_losses, label='mlpinit')
ax1.set_xlabel('epoch')
ax1.set_ylabel('loss')
ax1.legend()


ax2.plot(x, random_test_accs[:3], label='random')
ax2.plot(x, mlpinit_test_accs, label='mlpinit')
ax2.set_xlabel('epoch')
ax2.set_ylabel('accuracy')
ax2.legend()

plt.show()

In [ ]:
import numpy as np

In [ ]:
def find_best_speedup(random_accs, mlpinit_accs):
    best_speedup = 0
    best_line = 0
    for value in np.linspace(start=0, stop=1, num=10):
        first_achieved = 0
        second_achieved = 1000
        for j in range(len(random_accs)):
            if random_accs[j] > value:
                first_achieved = j
                break
        first_achieved += 1
        for j in range(len(mlpinit_accs)):
            if mlpinit_accs[j] > value:
                second_achieved = j
                break
        second_achieved += 1
        if second_achieved == 0:
            best_speedup = 0
        else:
            if best_speedup < first_achieved/second_achieved:
                best_line = value
                best_speedup = first_achieved/second_achieved
    return best_line, best_speedup

In [ ]:
best_line, best_speedup = find_best_speedup(random_test_accs[:3], mlpinit_test_accs)

In [ ]:
best_line, best_speedup = find_best_speedup(a1, a2)

In [ ]:
my_table = wandb.Table(columns=["best_line", "best_speedup"], data=[[best_line, best_speedup]])

In [ ]:
wandb.log({"table": my_table})

In [ ]:
a1 = [0.1, 0.2, 0.4]
a2 = [0.4, 0.4, 0.4]